## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-08-13-06-37 +0000,bbc,Starmer condemns Israel's plan to take over Ga...,https://www.bbc.com/news/articles/cm2kyg0g8kxo...
1,2025-08-08-13-00-06 +0000,nyt,What if You Can’t Believe the Official Numbers?,https://www.nytimes.com/2025/08/08/business/jo...
2,2025-08-08-12-59-53 +0000,nypost,Frightened Royal Caribbean passengers scream a...,https://nypost.com/2025/08/08/us-news/chaos-on...
3,2025-08-08-12-52-56 +0000,nyt,"A Starfish, a President and a Deep-Sea Frenzy ...",https://www.nytimes.com/2025/08/08/world/ameri...
4,2025-08-08-12-50-00 +0000,wsj,Israel’s Plan to Expand War Raises Fears for G...,https://www.wsj.com/world/middle-east/israels-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2323/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
49,trump,79
4,gaza,24
232,new,18
2,israel,15
599,tariffs,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
191,2025-08-07-21-50-00 +0000,wsj,President Trump said he would nominate Stephen...,https://www.wsj.com/politics/policy/stephen-mi...,135
241,2025-08-07-20-10-22 +0000,wapo,Trump celebrates new tariffs as negotiations c...,https://www.washingtonpost.com/business/2025/0...,124
323,2025-08-07-14-30-28 +0000,wapo,Trump’s new tariffs are hitting these major tr...,https://www.washingtonpost.com/business/2025/0...,123
280,2025-08-07-17-51-00 +0000,wsj,President Trump’s tariff rollout has taken num...,https://www.wsj.com/economy/trade/trump-tariff...,122
256,2025-08-07-19-02-30 +0000,wapo,Here’s where the money from Trump’s tariffs wi...,https://www.washingtonpost.com/business/2025/0...,121


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
191,135,2025-08-07-21-50-00 +0000,wsj,President Trump said he would nominate Stephen...,https://www.wsj.com/politics/policy/stephen-mi...
131,85,2025-08-08-03-37-58 +0000,wapo,Israel’s security cabinet approves plan for mi...,https://www.washingtonpost.com/world/2025/08/0...
258,62,2025-08-07-18-55-16 +0000,nypost,New NYC Council proposal will tie up businesse...,https://nypost.com/2025/08/07/us-news/new-nyc-...
152,42,2025-08-08-00-24-53 +0000,bbc,"Dozens of migrants detained under 'one in, one...",https://www.bbc.com/news/articles/ce35v0zyzvlo...
173,41,2025-08-07-22-40-47 +0000,nypost,Disney settles Gina Carano’s lawsuit after fir...,https://nypost.com/2025/08/07/media/disney-set...
239,39,2025-08-07-20-17-49 +0000,latimes,California Supreme Court sides with environmen...,https://www.latimes.com/environment/story/2025...
280,37,2025-08-07-17-51-00 +0000,wsj,President Trump’s tariff rollout has taken num...,https://www.wsj.com/economy/trade/trump-tariff...
281,37,2025-08-07-17-44-51 +0000,nypost,"Skydance CEO David Ellison feels the heat, mel...",https://nypost.com/2025/08/07/media/skydance-c...
171,36,2025-08-07-22-47-38 +0000,bbc,Trump calls for Intel boss to resign immediate...,https://www.bbc.com/news/articles/c70x6602pdyo...
45,32,2025-08-08-10-53-22 +0000,nyt,"Russian Strikes Kill 3 in Ukraine, Which May B...",https://www.nytimes.com/2025/08/08/world/europ...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
